In [4]:
import os
# import sys
# sys.path.append('../../src')
import pandas as pd
import numpy as np
from tqdm import tqdm
import random
from datetime import datetime
import time
from hydra import initialize, compose
import pathlib
# import config

import src.data_process.neg_sample as ng_sample
from src.data_process.utils import mix_merge
from src.data_process.data_split import data_split_user
from src.metrics.evaluate_ignite import CustomHR, CustomNDCG, CustomAuc_top, CustomAuc, CustomRecall_top, CustomPrecision_top
from src.model_entity import EntityCat
from src.data_utils import CatData
from src.utils.constants import DEFAULT_USER_COL,DEFAULT_ITEM_COL,DEFAULT_RATING_COL, DEFAULT_TIMESTAMP_COL


from sklearn import metrics, preprocessing
from sklearn.metrics import roc_auc_score, average_precision_score, recall_score, precision_score

import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

# import argparse
torch.manual_seed(0)

In [36]:
from ignite.engine import Engine, Events, create_supervised_trainer, create_supervised_evaluator, RemovableEventHandle
from ignite.metrics import Accuracy, Loss, Metric
from ignite.handlers import ModelCheckpoint, EarlyStopping
from ignite.exceptions import NotComputableError
from ignite.metrics.metric import sync_all_reduce, reinit__is_reduced
from ignite.contrib.handlers.tqdm_logger import ProgressBar
# from ignite.contrib.handlers import TensorboardLogger 
from ignite.contrib.handlers.wandb_logger import *

In [37]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [38]:
with initialize(version_base=None, config_path="../conf"):
    cfg = compose(config_name="config", overrides=[])

In [39]:
if device.type =='cpu':
    BATCH_SIZE = cfg.params.batch_size_cpu
    EPOCHS  = cfg.params.epochs_cpu
else:
    BATCH_SIZE = cfg.params.batch_size_gpu
    EPOCHS  = cfg.params.epochs_gpu

In [40]:
if device.type == 'cpu':
    use_amp=False
    df_train_pos  = ng_sample.read_feather(pathlib.Path(cfg.path.root, cfg.file.train_pos))
    df_train_neg = pd.read_feather(pathlib.Path(cfg.path.root, cfg.file.train_neg))
    df_test_ori = pd.read_feather(pathlib.Path(cfg.path.root, cfg.file.test)).iloc[:202,]
    df_all_features = pd.read_csv(pathlib.Path(cfg.path.root, cfg.file.all_features))
    df_train_pos = df_train_pos.sort_values(by=[DEFAULT_USER_COL]).iloc[:100,].reset_index(drop=True)
    df_train_neg = df_train_neg.sort_values(by=[DEFAULT_USER_COL]).iloc[:100*cfg.params.neg_train,].reset_index(drop=True)
else:
    use_amp=True
    df_train_pos  = ng_sample.read_feather(pathlib.Path(cfg.path.root, cfg.file.train_pos))
    df_train_neg = pd.read_feather(pathlib.Path(cfg.path.root, cfg.file.train_neg))
    df_test_ori = pd.read_feather(pathlib.Path(cfg.path.root, cfg.file.test))
    df_all_features = pd.read_csv(pathlib.Path(cfg.path.root, cfg.file.all_features))
    df_train_pos = df_train_pos.sort_values(by=[DEFAULT_USER_COL]).reset_index(drop=True)
    df_train_neg = df_train_neg.sort_values(by=[DEFAULT_USER_COL]).reset_index(drop=True)

In [41]:
df_train_pos[DEFAULT_RATING_COL] = 1

In [42]:
def concat_index(df1, df2):
    df2.index = df2.index//cfg.params.neg_train
    return pd.concat([df1, df2], axis=0).sort_index(kind='mregesort').reset_index(drop=True)

In [43]:
df_train_all = concat_index(df_train_pos, df_train_neg)

In [44]:
df_train_all['flag'] = 1
df_test_ori['flag'] = 0
df_all = pd.concat([df_train_all, df_test_ori], axis=0).reset_index(drop=True)

user features: 
       'WindowID_user', 'Split', 'City',
       'State', 'Country', 'Zip_user', 'DegreeType', 'Major', 'GraduationDate',
       'WorkHistoryCount', 'TotalYearsExperience', 'CurrentlyEmployed',
       'ManagedOthers', 'ManagedHowMany',
       
job features: 
       'WindowID_job', 'City_job',
       'State_job', 'Country_job', 'Zip_job', 'StartDate', 'EndDate',

### Choose the features and process data for the training

In [64]:
df_all.columns

Index(['userid', 'itemid', 'rating', 'flag'], dtype='object')

In [65]:
user_features = []
user_features_extend = [DEFAULT_USER_COL] + user_features

item_features = []
item_features_extend =[DEFAULT_ITEM_COL] + item_features

base_features = [DEFAULT_USER_COL, DEFAULT_ITEM_COL, DEFAULT_RATING_COL]

In [66]:
df_mix_merge = mix_merge(df_all , df_all_features, user_features_extend, item_features_extend)

In [67]:
def _cat_encode(df_data, list_f, encoder):
    for f in list_f:
        df_data[f] = encoder.fit_transform(df_data[f].astype('category').cat.codes.values)
    return df_data

In [68]:
def _embedding_dimension(df_all_encode, features_to_train, max_dim=50):

    embedding_size = []
    features_to_em = [i for i in features_to_train if i !=DEFAULT_RATING_COL]
    for c in features_to_em:
        num_unique_values = int(df_all_encode[c].nunique())
        embed_dim = int(min(np.ceil(num_unique_values/2), max_dim))
        embedding_size.append([num_unique_values, embed_dim])  
    return embedding_size

In [69]:
def encode_data(df_mix_merge, features_to_code, features_to_train, max_dim=50):
    encoder = preprocessing.LabelEncoder()
    df_all_encode = _cat_encode(df_mix_merge, features_to_code, encoder)
    df_train = df_all_encode[df_all.flag==1]
    df_test = df_all_encode[df_all.flag==0]
    df_train = df_train[features_to_train]
    df_test = df_test[features_to_train]
    embedding_size = _embedding_dimension(df_all_encode, features_to_train, max_dim)
    return df_train, df_test, embedding_size

In [70]:
num_feature=[]
features_to_code = df_mix_merge.columns
features_to_train = [DEFAULT_USER_COL, DEFAULT_ITEM_COL]+ user_features + item_features +[DEFAULT_RATING_COL]


In [71]:
df_train,  df_test, embedding_size = encode_data(df_mix_merge, features_to_code, features_to_train, max_dim=cfg.params.emb_dim)

print(f'The size of embedding layers:{embedding_size}')

The size of embedding layers:[[36, 18], [413, 50], [35, 18], [19, 10], [36, 18], [230, 50], [39, 20], [1, 1], [245, 50]]


## Run data check before training 

Check the ratio of positive and negative samples

In [72]:
assert len(df_train[df_train.rating==0])/len(df_train[df_train.rating==1]) == cfg.params.neg_train, 'wrong neg/pos ratio in training set'
assert len(df_test[df_test.rating==0])/len(df_test[df_test.rating==1]) == cfg.params.neg_test, 'wrong neg/pos ratio in test set '
#Check if all the users in test can be found in training set
assert sum(np.isin(df_test.userid.unique(), df_train.userid.unique(), assume_unique=True)) == len(df_test.userid.unique()), 'cold start'
#The the uniqueness of items between training and test. For a user, on common items between training and test dataset. 
assert df_all.shape[0] ==df_train.shape[0]+df_test.shape[0], 'wrong data concat'
assert sum(df_all.groupby(['userid']).apply(lambda x: len(x['itemid'].unique()))) == df_all.shape[0], 'train and test have overlap item'

## Creat the numpy array for training 

In [73]:
df_train_split, df_val_split = data_split_user(df_train, val_size=0.2)

np_train = df_train_split.values
np_val = df_val_split.values
np_test = df_test.values

In [74]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

In [75]:
train_dataset = CatData(np_train)
val_dataset = CatData(np_val)
test_dataset = CatData(np_test) 
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0,  worker_init_fn=seed_worker,generator=g)
val_loader = data.DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False, num_workers=0,  worker_init_fn=seed_worker,generator=g)
test_loader = data.DataLoader(test_dataset, batch_size=cfg.params.neg_test+1, shuffle=False, num_workers=0,worker_init_fn=seed_worker,generator=g )

In [76]:
model = EntityCat(embedding_size = embedding_size, num_numerical_cols = len(num_feature),
               output_size = 1)
model.to(device)

EntityCat(
  (all_embeddings): ModuleList(
    (0): Embedding(36, 18)
    (1): Embedding(413, 50)
    (2): Embedding(35, 18)
    (3): Embedding(19, 10)
    (4): Embedding(36, 18)
    (5): Embedding(230, 50)
    (6): Embedding(39, 20)
    (7): Embedding(1, 1)
    (8): Embedding(245, 50)
  )
  (mlp_layers): Sequential(
    (0): Linear(in_features=235, out_features=100, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.4, inplace=False)
  )
  (predict_layer): Linear(in_features=100, out_features=1, bias=True)
)

In [77]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=cfg.params.lr)

## Model the training 

In [78]:
def output_trans_loss(output):
    return output['y_pred'], output['label']

val_metrics_train = {
    'auc': CustomAuc(),
    "loss": Loss(criterion, output_transform=output_trans_loss)
}

val_metrics_test = {
    'hr': CustomHR(),
    'ndcg': CustomNDCG(k=cfg.params.topk),
    'auc': CustomAuc(),
    'auc_top': CustomAuc_top(),
    'recall_top': CustomRecall_top(k=cfg.params.topk),
    'precision_top': CustomPrecision_top(k=cfg.params.topk),
    "loss": Loss(criterion, output_transform=output_trans_loss)
}

In [79]:
scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
def train_step(engine, batch):
    model.train()
    optimizer.zero_grad()
    x, y = batch[0].to(device), batch[1].to(device)
    with torch.cuda.amp.autocast(enabled=use_amp):
        y_pred = model(x).reshape(1,-1).flatten()
        loss = criterion(y_pred, y.float())
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()
    return loss.item()

def validation_step(engine, batch):
    model.eval()
    with torch.no_grad():
        x, label = batch[0].to(device), batch[1].to(device)
        y_pred = model(x).reshape(1,-1).flatten()
        label=label.float()
        return {'label':label, 'y_pred':y_pred}


def test_step(engine, batch):
    model.eval()
    with torch.no_grad():
        x, label = batch[0].to(device), batch[1].to(device)
        y_pred = model(x).reshape(1,-1).flatten()
        label=label.float()
        
        y_pred_top, indices = torch.topk(y_pred, engine.state.topk)
        
        y_pred_top = y_pred_top.detach().cpu().numpy()
        reco_item = torch.take(x[:,1], indices).cpu().numpy().tolist()
        pos_item = x[0,1].cpu().numpy().tolist()  # ground truth, item id
        label_top = label[indices].cpu().numpy()
        indices = indices.cpu().numpy()
        return {'pos_item':pos_item, 'reco_item':reco_item, 'y_pred_top':y_pred_top, 
                'label_top':label_top, 'label':label, 'y_pred':y_pred, 'y_indices':indices}
    
trainer = Engine(train_step)

train_evaluator = Engine(validation_step)
# train_evaluator.state_dict_user_keys.append('topk')

val_evaluator = Engine(validation_step)
# val_evaluator.state_dict_user_keys.append('topk')

test_evaluator = Engine(test_step)
test_evaluator.state_dict_user_keys.append('topk')

# @val_evaluator.on(Events.STARTED)
# def init_user_value():
#     val_evaluator.state.topk=3
    
# @train_evaluator.on(Events.STARTED)
# def init_user_value():
#     train_evaluator.state.topk=3

@train_evaluator.on(Events.STARTED)
def init_user_value():
    test_evaluator.state.topk=cfg.params.topk
    
    
# Attach metrics to the evaluators
for name, metric in val_metrics_train.items():
    metric.attach(train_evaluator, name)

for name, metric in val_metrics_train.items():
    metric.attach(val_evaluator, name)

    
for name, metric in val_metrics_test.items():
    metric.attach(test_evaluator, name)
    
# Eearly_stop 
def score_function(engine):
    val_loss = engine.state.metrics['auc']
    return val_loss

Eearly_stop_handler = EarlyStopping(patience=cfg.params.patience, score_function=score_function, trainer=trainer)
val_evaluator.add_event_handler(Events.COMPLETED, Eearly_stop_handler)

In [80]:
@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    train_evaluator.run(train_loader)
    metrics = train_evaluator.state.metrics
    auc = metrics['auc']
    loss = metrics['loss']
    print(f'Training Results- Epoch[{trainer.state.epoch}]  Avg loss: {loss:.2f} \
          Avg auc:{auc:.2f}')


@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    val_evaluator.run(val_loader)
    metrics = val_evaluator.state.metrics
    auc = metrics['auc']
    loss = metrics['loss']
    print(f'Validation Results- Epoch[{trainer.state.epoch}]  Avg loss: {loss:.2f} \
          Avg auc:{auc:.2f}')

    
@trainer.on(Events.COMPLETED)
def log_test_results(trainer):
    test_evaluator.run(test_loader)
    metrics = test_evaluator.state.metrics
    hr = metrics['hr']
    ndcg = metrics['ndcg']
    auc = metrics['auc']
    auc_top = metrics['auc_top']
    recall = metrics['recall_top']
    precision = metrics['precision_top']
    loss = metrics['loss']
    print(f"Test Results - Epoch[{trainer.state.epoch}]  Avg loss: {loss:.2f} \
     Avg ndcg: {ndcg:.2f}  Avg auc: {auc:.2f}  Avg auc_top: {auc_top:.2f} \
      Avg recall: {recall:.2f}  Avg precision: {precision:.2f}")

pbar = ProgressBar(persist=False)
pbar.attach(trainer)

In [81]:
# trainer.run(train_loader, max_epochs=2)

In [82]:
config_dict = dict(cfg.params)
config_dict['Features']='-'.join(user_features+item_features)
wandb_logger = WandBLogger(
    project="pytorch-jrs",
    name="-".join(user_features)+'-'+'-'.join(item_features),
    config=config_dict,
    tags=["entity", "jrs"]
)

to_save = {'model': model}
checkpoint_handler = ModelCheckpoint(
    wandb_logger.run.dir,
    n_saved=1, filename_prefix='best',
    score_name="auc",
    global_step_transform=global_step_from_engine(trainer)
)

val_evaluator.add_event_handler(Events.COMPLETED, checkpoint_handler, to_save)

    
wandb_logger.attach_output_handler(
    trainer,
    event_name=Events.ITERATION_COMPLETED,
    tag="training",
    output_transform=lambda loss: {"loss": loss}
)

wandb_logger.attach_output_handler(
    train_evaluator,
    event_name=Events.EPOCH_COMPLETED,
    tag="training",
    metric_names=['loss','auc'],
    global_step_transform=lambda *_: trainer.state.iteration,
)

wandb_logger.attach_output_handler(
    val_evaluator,
    event_name=Events.EPOCH_COMPLETED,
    tag="validation",
    metric_names=['loss',"auc"],
    global_step_transform=lambda *_: trainer.state.iteration,
)


wandb_logger.attach_output_handler(
    test_evaluator,
    event_name=Events.COMPLETED,
    tag="test",
    metric_names=['loss',"auc", 'hr', 'ndcg', 'auc_top', 'recall_top', 'precision_top'],
    global_step_transform=lambda *_: trainer.state.iteration,
)


wandb_logger.attach_opt_params_handler(
    trainer,
    event_name=Events.ITERATION_STARTED,
    optimizer=optimizer,
    param_name='lr'  # optional
)

# wandb_logger.watch(model) 

trainer.run(train_loader, max_epochs=EPOCHS)
wandb_logger.close()

/Users/hao/.pyenv/versions/3.6.15/envs/torch-cpu5/lib/python3.6/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


Training Results- Epoch[1]  Avg loss: 0.85           Avg auc:0.92
Validation Results- Epoch[1]  Avg loss: 0.85           Avg auc:0.72


Training Results- Epoch[2]  Avg loss: 0.79           Avg auc:0.96
Validation Results- Epoch[2]  Avg loss: 0.81           Avg auc:0.74


Training Results- Epoch[3]  Avg loss: 0.70           Avg auc:0.97
Validation Results- Epoch[3]  Avg loss: 0.74           Avg auc:0.75


Epoch [4/20]: [18/19]  95%|█████████▍ [00:00<00:00]

...

Epoch [19/20]: [18/19]  95%|█████████▍ [00:00<00:00]

Training Results- Epoch[19]  Avg loss: 0.62           Avg auc:1.00


Validation Results- Epoch[19]  Avg loss: 0.73           Avg auc:0.81


Training Results- Epoch[20]  Avg loss: 0.62           Avg auc:1.00
Validation Results- Epoch[20]  Avg loss: 0.73           Avg auc:0.81
Test Results - Epoch[20]  Avg loss: 0.89      Avg ndcg: 0.00  Avg auc: 0.51  Avg auc_top: 0.00       Avg recall: 0.00  Avg precision: 0.00


lr/group_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test/auc,▁
test/auc_top,▁
test/hr,▁
test/loss,▁
test/ndcg,▁
test/precision_top,▁
test/recall_top,▁
training/auc,▁▄▅▇▇███████████████
training/loss,██▇▆▆▅▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation/auc,▁▂▃▄▅▅▇▇▇███████████


## Quality evaluation 
display the recommended items for the user. Sample the recommended test users 

In [281]:
unique_users_test = df_test[DEFAULT_USER_COL].unique()
# np.random.seed(123)
sub_users_test = np.random.choice(unique_users_test, 5, replace=False)
sub_users_test

array([53693, 31989, 72002, 33852,  1441])

In [282]:
df_test_sub = df_test[df_test[DEFAULT_USER_COL].isin(sub_users_test)]
# df_test_sub

In [287]:
np_test_sub = df_test_sub.values

In [288]:
test_dataset_sample = CatData(np_test_sub)
test_sample_loader = data.DataLoader(test_dataset_sample, batch_size=cfg.params.neg_test+1, shuffle=False, num_workers=0,worker_init_fn=seed_worker,generator=g)

In [289]:
class RecoList(Metric):
    '''
    The recommendation list 
    '''
    def __init__(self, output_transform=lambda x: [x['pos_item'], x['reco_item'], x['userid'], x['y_pred_top'], x['indices']], device="cpu"):
#         self._reco_dict = []
        self.item= np.array([])
        self.userid= np.array([])
        self.prob= np.array([])
        self.indices = []
#         self.cnt = 0
        super(RecoList, self).__init__(output_transform=output_transform, device=device)

    @reinit__is_reduced
    def reset(self):
        self.item= np.array([])
        self.userid= np.array([])
        self.prob= np.array([])
#         self.cnt = 0
        super(RecoList, self).reset()

    @reinit__is_reduced
    def update(self, output):
        gt_item = output[0]
        reco_item = output[1]
        userid = output[2]
        prob = output[3]
        indices = output[4]
        self.item = np.append(self.item, reco_item)
        self.userid = np.append(self.userid, userid)
        self.prob = np.append(self.prob, prob)
        self.indices.append(indices)
        
    @sync_all_reduce("_reco_dict")
    def compute(self):
        return self.indices, self.prob

In [290]:
val_metrics_test_quanlity = {
    'recolist': RecoList()
}

In [291]:
def test_q_step(engine, batch):
    model.eval()
    with torch.no_grad():
        x, label = batch[0].to(device), batch[1].to(device)
        y_pred = model(x).reshape(1,-1).flatten()
#         print(y_pred)
        label=label.float()
        y_pred_top, indices = torch.topk(y_pred, engine.state.topk)
        y_pred_top = y_pred_top.cpu().numpy().tolist()
        y_pred_top = y_pred_top
        reco_item = torch.take(x[:,1], indices).cpu().numpy().tolist()
        pos_item = x[0,1].cpu().numpy().tolist()  # ground truth, item id 
        userid = x[:engine.state.topk,0].cpu().numpy().tolist()
        return {'reco_item': reco_item, 'pos_item':pos_item, 'userid':userid, 'y_pred_top':y_pred_top, 'indices':indices.cpu().numpy().tolist()}

In [292]:
test_q_evaluator=Engine(test_q_step)
for name, metric in val_metrics_test_quanlity.items():
    metric.attach(test_q_evaluator, name)

In [293]:
test_q_evaluator.state_dict_user_keys.append('topk')

@test_q_evaluator.on(Events.STARTED)
def init_user_value():
    test_q_evaluator.state.topk=cfg.params.topk

In [294]:
test_q_evaluator.run(test_sample_loader)

State:
	iteration: 5
	epoch: 1
	epoch_length: 5
	max_epochs: 1
	output: <class 'dict'>
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: <class 'torch.utils.data.dataloader.DataLoader'>
	seed: <class 'NoneType'>
	times: <class 'dict'>
	topk: 10

In [295]:
reco_indices, prob =test_q_evaluator.state.metrics['recolist']

In [296]:
reco_indices

[[0, 96, 81, 93, 66, 85, 77, 90, 51, 86],
 [0, 62, 43, 17, 82, 25, 29, 49, 67, 35],
 [66, 96, 0, 81, 93, 75, 77, 85, 62, 71],
 [75, 77, 76, 93, 85, 0, 59, 42, 81, 96],
 [76, 77, 0, 93, 85, 59, 42, 81, 96, 62]]

In [304]:
reco_indices_extend = []
for i in range(len(reco_indices)):
    reco_indices_extend += [(cfg.params.neg_test+1)*(i)+j for j in reco_indices[i]]

In [305]:
# df_test_sub.iloc[reco]

In [306]:
df_reco = df_test_sub.iloc[reco_indices_extend].copy()

In [309]:
# df_reco

In [310]:
df_reco['prob'] = prob

In [171]:
job_info = pd.read_csv('../../data/jobs/jobid_title.csv')

In [172]:
job_info.rename(columns={'item_id':'itemid'}, inplace=True)

In [173]:
# job_info

In [312]:
pd.merge(df_reco, job_info, how='left', on='itemid')

,userid,itemid,rating,prob,Unnamed: 0,Title
0,1441,26715,1,0.990982,979738,Truck Driver- CDL
1,1441,126099,0,0.978732,595954,Data Analyst
2,1441,57477,0,0.016489,447313,"Sears HAS Store Manager_Clifton,NJ (JC)"
3,1441,46944,0,0.014807,726375,Training and Application Engineer
4,1441,128873,0,0.009509,990232,C-12 Avionics Technician - ROVER
5,1441,115613,0,0.004872,31673,Thin Film Process Engineer
6,1441,114149,0,0.004324,30451,Carpenter/Crater
7,1441,51852,0,0.004047,866714,Sales - Insurance - Account Executive
8,1441,98719,0,0.003649,25546,Layout Technician
9,1441,31263,0,0.003600,864560,Hair Stylist & Nail Techs Full/Part Time Sig...
